In [9]:
%load_ext autoreload
%autoreload 2

import pyscan as ps
from pyscan.drivers.testing.auto_test_driver import test_driver
from pyvisa import ResourceManager, VisaIOError
import pytest
from time import sleep

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
rm = ResourceManager()

rs = rm.list_resources()
# print(rs)
for r in rs:
    res = rm.open_resource(r)
    try:      
        name = res.query('*IDN?')
        if 'SR830' in name:
            print(r, name)
            break
    except VisaIOError:
        pass
    res.close()

GPIB0::8::INSTR Stanford_Research_Systems,SR830,s/n86813,ver1.07 



In [11]:
name

'Stanford_Research_Systems,SR830,s/n86813,ver1.07 \n'

In [18]:
srs830 = ps.Stanford830(res)

test_driver(srs830, verbose=False)

Blacklisted settings that will not be tested or changed are: 
[('_amplitude', 0.496), ('_input_configuration', 'A-B'), ('_time_constant', 30)]

Beginning tests for:  Stanford830  version  1.0.1

7 range properties found and tested out of 25 total settings found.
0 values properties found and tested out of 25 total settings found.
12 indexed values properties found and tested out of 25 total settings found.
2 dict values properties found and tested out of 25 total settings found.
3 blacklisted settings not testing (likely due to interdependencies not suitable for automated testing)
21 properties tested out of 25 total settings.

Restored settings are different for the following:  {('id', 'Stanford_Research_Systems,SR830,s/n86813,ver1.07 ')}


 Driver tests passed, instrument: Stanford830 looks ready to go. 
The new test log for this driver is:  Passed with stanford830 version v1.0.1 tested on pyscan version v0.7.3 at 2024-09-26 11:19:53
 Driver test results logged. 
Checking driver doc 

AssertionError: Attribute 'buffer_points' listed in docstring but not the driver.

In [ ]:
# Test get and set display

srs830.set_display(1, 'r', 'aux1') # number, source, ratio
source, ratio = srs830.get_display(1)
assert source == 'r'
assert ratio == 'aux1'


srs830.set_display(1, 'x', 'none')
source, ratio = srs830.get_display(1)
assert source == 'x'
assert ratio == 'none'


with pytest.raises(AssertionError):
    srs830.set_display(1, 'y', 'none')
print('Sucessfully failed setting channel 1 to y')
    
    
srs830.set_display(2, 'theta', 'aux3')
source, ratio = srs830.get_display(2)
assert source == 'theta'
assert ratio == 'aux3'


srs830.set_display(2, 'y', 'none')
source, ratio = srs830.get_display(2)
assert source == 'y'
assert ratio == 'none'


with pytest.raises(AssertionError):
    srs830.set_display(2, 'x', 'none')
print('Sucessfully failed setting channel 2 to x')

print("Successfully tested get and set display")


Sucessfully failed setting channel 1 to y
Sucessfully failed setting channel 2 to x
Successfully tested get and set display


In [ ]:
# Test get and set offset and expand

srs830.set_channel_offset_expand(1, 100, 10)
print(srs830.get_channel_offset_expand(1))

srs830.set_channel_offset_expand(1, 0, 1)
print(srs830.get_channel_offset_expand(1))


['0', '1']
['0', '0']


In [ ]:
# Test auto offset
# v0 should be some value, v_offset should be close to 0
srs830.set_channel_offset_expand(1, 0, 1)
v0 = srs830.read('x')
srs830.auto_offset('x')
v_offset = srs830.read('x')
print(v0, v_offset)
assert abs(v_offset - 0) < 0.0001


srs830.set_channel_offset_expand(2, 0, 1)
v0 = srs830.read('y')
srs830.auto_offset('y')
v_offset = srs830.read('y')
print(v0, v_offset)
assert abs(v_offset - 0) < 0.0001

srs830.set_channel_offset_expand(1, 0, 1)
v0 = srs830.read('r')
srs830.auto_offset('r')
v_offset = srs830.read('r')
print(v0, v_offset)
assert abs(v_offset - 0) < 0.0001


srs830.set_channel_offset_expand(1, 0, 1)
srs830.set_display(1, 'x', 'none')


2.18428e-09 2.18428e-09
-2.18439e-09 -2.18439e-09
2.18428e-09 2.18428e-09


In [ ]:
# Test read aux inputs
# shoudl show some random small value
for i in range(1, 5):
    aux_input = srs830.read_aux_input(i)
    print(aux_input)
    assert abs(aux_input - 0) < 0.1

0.00466667
-0.001
0.00166667
0.00233333


In [ ]:
# Read x, y, r, theta

print(srs830.read('x'))
print(srs830.read('y'))
print(srs830.read('r'))
print(srs830.read('theta'))

2.18428e-09
-2.18439e-09
2.18428e-09
-2597.219879105803


In [ ]:
# Read display

for i in range(1, 3):
    print(srs830.read_display(i))

2.18439e-09
-2.18451e-09


In [ ]:
# Test snap

for source in ['y', 'r', 'theta', 'aux1', 'aux2', 'aux3',
                  'aux4', 'frequency', 'display1', 'display2']:
    print(srs830.snap('x', source))

srs830.snap('x', 'y', 'r', 'theta', 'frequency', 'display1')


with pytest.raises(AssertionError):
    srs830.snap('x', 'y', 'r', 'theta', 'frequency', 'display1', 'display2')
    # snap only accepts 6 entries

[2.18428e-09, -2.18439e-09]
[2.18428e-09, 2.18428e-09]
[2.18428e-09, -2670.7877221406993]
[2.18428e-09, -0.003]
[2.18428e-09, -0.00566667]
[2.18428e-09, -0.008]
[2.18428e-09, -0.00833333]
[2.18428e-09, 1000.0]
[2.18428e-09, 2.18439e-09]
[2.18428e-09, -2.18451e-09]


In [ ]:
# test get ID

id = srs830.get_identificaiton_string()
assert id == 'Stanford_Research_Systems,SR830,s/n86813,ver1.07 '
print(id)

Stanford_Research_Systems,SR830,s/n86813,ver1.07 


In [ ]:
# test buffer_points

srs830.buffer_points
srs830.wait_for_trigger()
srs830.set_buffer_mode(0.25)
srs830.wait_for_trigger()
# srs830.start()
srs830.trigger()
sleep(20)
srs830.buffer_points


AttributeError: 'Stanford830' object has no attribute 'buffer_points'

In [ ]:
srs830.query('SRAT?')

'0\n'

In [ ]:
srs830.sample_rate

0.0625